In [4]:
import os

import pandas as pd
# OpenAI API 키 설정 (이 예제에서는 사용되지 않음)
api_key = os.getenv("OPENAI_API_KEY")

df = pd.read_csv("Titanic.csv")

print(df.shape)
print(df.columns.tolist())

(891, 12)
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


SQL<BR>
CSV 데이터와 상호작용할 때 SQL을 사용하는 것이 권장됩니다. 이는 임의의 Python 코드보다 권한을 제한하고 쿼리를 정제하기가 더 쉽기 때문입니다.<br>

대부분의 SQL 데이터베이스는 CSV 파일을 테이블로 쉽게 로드할 수 있습니다 (DuckDB, SQLite 등). 이렇게 하면 SQL 사용 사례 가이드에 설명된 모든 체인 및 에이전트 생성 기술을 사용할 수 있습니다. 아래는 SQLite를 사용하는 간단한 예입니다:

In [5]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///titanic.db")
df.to_sql("titanic", engine, index=False)

891

In [6]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM titanic WHERE Age < 2;")

sqlite
['titanic']


'[(79, 1, 2, \'Caldwell, Master. Alden Gates\', \'male\', 0.83, 0, 2, \'248738\', 29.0, None, \'S\'), (165, 0, 3, \'Panula, Master. Eino Viljami\', \'male\', 1.0, 4, 1, \'3101295\', 39.6875, None, \'S\'), (173, 1, 3, \'Johnson, Miss. Eleanor Ileen\', \'female\', 1.0, 1, 1, \'347742\', 11.1333, None, \'S\'), (184, 1, 2, \'Becker, Master. Richard F\', \'male\', 1.0, 2, 1, \'230136\', 39.0, \'F4\', \'S\'), (306, 1, 1, \'Allison, Master. Hudson Trevor\', \'male\', 0.92, 1, 2, \'113781\', 151.55, \'C22 C26\', \'S\'), (382, 1, 3, \'Nakid, Miss. Maria ("Mary")\', \'female\', 1.0, 0, 2, \'2653\', 15.7417, None, \'C\'), (387, 0, 3, \'Goodwin, Master. Sidney Leonard\', \'male\', 1.0, 5, 2, \'CA 2144\', 46.9, None, \'S\'), (470, 1, 3, \'Baclini, Miss. Helene Barbara\', \'female\', 0.75, 2, 1, \'2666\', 19.2583, None, \'C\'), (645, 1, 3, \'Baclini, Miss. Eugenie\', \'female\', 0.75, 2, 1, \'2666\', 19.2583, None, \'C\'), (756, 1, 2, \'Hamalainen, Master. Viljo\', \'male\', 0.67, 1, 1, \'250649\', 

In [7]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [8]:
agent_executor.invoke({"input": "what's the average age of survivors"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`



CREATE TABLE titanic (
	"PassengerId" BIGINT, 
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"SibSp" BIGINT, 
	"Parch" BIGINT, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/
Invoking: `sql_db_query` with `{'query': 'SELECT AVG(Age) AS Average_Age FROM titanic WHERE Survived = 1'}`


[(28.343689655172415,)]The average age of survivors in the Titanic dataset is approximately 28.34 years.

> Finished chain.


{'input': "what's the average age of survivors",
 'output': 'The average age of survivors in the Titanic dataset is approximately 28.34 years.'}

Pandas<br>
SQL 대신 Pandas와 같은 데이터 분석 라이브러리와 LLM의 코드 생성 기능을 사용하여 CSV 데이터와 상호작용할 수도 있습니다. 이 접근 방식은 광범위한 보호 조치가 없는 한 프로덕션 용도로 적합하지 않습니다. 이 때문에 코드 실행 유틸리티와 생성자는 langchain-experimental 패키지에 있습니다.<Br>

체인<br>
대부분의 LLM은 충분한 Pandas Python 코드를 학습하여 요청만 하면 이를 생성할 수 있습니다:

In [9]:
ai_msg = llm.invoke(
    "I have a pandas DataFrame 'df' with columns 'Age' and 'Fare'. Write code to compute the correlation between the two columns. Return Markdown for a Python code snippet and nothing else."
)
print(ai_msg.content)

```python
df['Age'].corr(df['Fare'])
```


In [10]:
correlation = df['Age'].corr(df['Fare'])
correlation

0.09606669176903888

이 기능을 Python 실행 도구와 결합하여 간단한 데이터 분석 체인을 만들 수 있습니다. 먼저 CSV 테이블을 데이터프레임으로 로드하고 이 데이터프레임에 대한 접근 권한을 도구에 부여합니다:

In [12]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

df = pd.read_csv("Titanic.csv")
tool = PythonAstREPLTool(locals={"df": df})
tool.invoke("df['Fare'].mean()")

32.204207968574636

In [13]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
llm_with_tools.invoke(
    "I have a dataframe 'df' and want to know the correlation between the 'Age' and 'Fare' columns"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_D93e4kRTgcVyJ8Qa6Vm0jVyx', 'function': {'arguments': '{"query":"df[[\'Age\', \'Fare\']].corr()"}', 'name': 'python_repl_ast'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 125, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c6fc5e2f-9a30-4cd7-879f-aed371f5dcb5-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "df[['Age', 'Fare']].corr()"}, 'id': 'call_D93e4kRTgcVyJ8Qa6Vm0jVyx'}])

에이전트<BR>
복잡한 질문의 경우 LLM이 이전 실행의 입력 및 출력을 유지하면서 코드를 반복적으로 실행할 수 있는 기능이 도움이 될 수 있습니다. 여기서 에이전트가 등장합니다. 에이전트는 LLM이 도구를 몇 번이나 호출해야 하는지 결정하고 지금까지 수행한 실행을 추적할 수 있게 해줍니다. create_pandas_dataframe_agent는 데이터프레임을 쉽게 다룰 수 있도록 하는 내장 에이전트입니다.








In [17]:
from langchain_experimental.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm, df, agent_type="openai-tools", verbose=True)
agent.invoke(
    {
        "input": "What's the correlation between age and fare? is that greater than the correlation between fare and survival?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[['Age', 'Fare']].corr()"}`


           Age      Fare
Age   1.000000  0.096067
Fare  0.096067  1.000000
Invoking: `python_repl_ast` with `{'query': "df[['Fare', 'Survived']].corr()"}`


              Fare  Survived
Fare      1.000000  0.257307
Survived  0.257307  1.000000The correlation between age and fare is 0.096067, while the correlation between fare and survival is 0.257307. Therefore, the correlation between fare and survival is greater than the correlation between age and fare.

> Finished chain.


{'input': "What's the correlation between age and fare? is that greater than the correlation between fare and survival?",
 'output': 'The correlation between age and fare is 0.096067, while the correlation between fare and survival is 0.257307. Therefore, the correlation between fare and survival is greater than the correlation between age and fare.'}

여러 CSV 파일<br>
여러 CSV 파일(또는 데이터프레임)을 처리하려면 Python 도구에 여러 데이터프레임을 전달하기만 하면 됩니다. create_pandas_dataframe_agent 생성자는 이를 기본적으로 처리할 수 있으며, 하나의 데이터프레임 대신 데이터프레임 목록을 전달할 수 있습니다. 체인을 직접 구성하려면 다음과 같이 할 수 있습니다:

In [23]:
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(tool.name, first_tool_only=True)

df_1 = df[["Age", "Fare"]]
df_2 = df[["Fare", "Survived"]]

tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2})
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
df_template = """```python
{df_name}.head().to_markdown()
>>> {df_head}
```"""
df_context = "\n\n".join(
    df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
    for _df, df_name in [(df_1, "df_1"), (df_2, "df_2")]
)

system = f"""You have access to a number of pandas dataframes. \
Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

{df_context}

Given a user question about the dataframes, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Make sure to refer only to the variables mentioned above."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

chain = prompt | llm_with_tool | parser | tool
chain.invoke(
    {
        "question": "return the difference in the correlation between age and fare and the correlation between fare and survival"
    }
)

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given